In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install pomegranate

     |████████████████████████████████| 17.9 MB 118 kB/s 
     |████████████████████████████████| 15.7 MB 58 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.21.3 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [1]:
!pip install dipy

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import plotly_express as px
# px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")

In [3]:
road_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/route_network/clustered'
trips_cluster_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.2_wayid_on_trips'
train_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/model/3.2'

In [ ]:
# data_path = '/content/drive/My Drive/Colab Notebooks/RoutePredict/Data'
# module_path = '/content/drive/My Drive/Colab Notebooks/RoutePredict/Module'

# cross validation

In [4]:
dfs = []
day_len = []
day_trip_count = []
# days = range(1,32)
days = range(1,2)
# months = range(1,4)
months = range(1,2)

for month in months:
  for day in days:
    try:
      sp_seq_df = pd.read_csv(trips_cluster_data_path + f'/lebeled/testtrips_2019_{month}_{day}.csv', parse_dates=['time_stamp'])
    except:
      pass

    sp_seq_df = sp_seq_df.sort_values(by=['trip_id', 'time_stamp'])

    trip_count = sp_seq_df.trip_id.value_counts()
    trip_count = trip_count[trip_count >= 12]
    sp_seq_df = sp_seq_df[sp_seq_df.trip_id.isin(trip_count.index)]
    sp_seq_df['month'] = month
    sp_seq_df['day'] = day
    day_len.append(len(sp_seq_df))
    day_trip_count.append(len(trip_count))
    dfs.append(sp_seq_df)
print(day_len)
print(day_trip_count)
df = pd.concat(dfs, ignore_index=True)
df

# df = df[['trip_id', 'way_id', 'day', 'month','route_cluster']]
df.head()

[12599]
[356]


,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,way_id,dist,route_cluster,month,day
0,2019-01-01 00:00:39,016000200000352253062499073,17.49378,102.79729,0,6.0,1,686967969,0.015018,1.0,1,1
1,2019-01-01 04:26:59,016000200000352253062499073,17.53611,102.75656,53,6.0,1,686957536,0.004565,9.0,1,1
2,2019-01-01 04:29:39,016000200000352253062499073,17.54386,102.73553,54,6.0,1,686912257,0.001649,10.0,1,1
3,2019-01-01 04:32:39,016000200000352253062499073,17.55281,102.71174,56,6.0,1,686911019,0.011723,10.0,1,1
4,2019-01-01 04:35:59,016000200000352253062499073,17.56263,102.68405,54,6.0,1,686910206,0.014603,10.0,1,1


In [5]:
len(pd.unique(df.way_id))

1776

## get rid of mini cluster

In [15]:
import collections
import pickle

# pkl_filename = train_path+"/QBcluster_test_way_treshold12.pkl"
pkl_filename = train_path+"/QBcluster_test_way_treshold24.pkl"
with open(pkl_filename, 'rb') as file:
    clusters = pickle.load(file)

print("Nb. clusters:", len(clusters))
print("Cluster sizes:", map(len, clusters))
print("Small clusters:", collections.Counter(clusters < 2)[True], end=' ')
print("Small clusters:", clusters < 2)
print("Streamlines indices of the first cluster:\n", clusters[0].indices)
# print("Centroid of the last cluster:\n", clusters[-1].centroid)

is_mini_cluster = (clusters == 1)

mini_cluster = []
for ci in range(0, len(is_mini_cluster)):
  if is_mini_cluster[ci]:
    mini_cluster.append(ci)
print(len(mini_cluster)) 

df.loc[df.route_cluster.isin(mini_cluster), 'route_cluster'] = -1
df.loc[df.route_cluster == -1].way_id.value_counts()

Nb. clusters: 547
Cluster sizes: <map object at 0x7f4598046850>
Small clusters: 241 Small clusters: [False False False False False False False False False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False  True False False False False False False False False
 False False False False False False False False False  True False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False  True False False False False False False
 False False False False False False False False False False  True False
 False False False False  True False False False False False False False
  True False  True  True False False  True False  True False False False
 False False False False

82585529     77
82374801     64
775708651    55
43344864     30
150586957    26
             ..
938366234     1
117824530     1
906768967     1
110678834     1
383231162     1
Name: way_id, Length: 1183, dtype: int64

## X and Label shuffle

In [16]:
state_names = []
uniq_r_cluster = df.route_cluster.unique()
for cluster in uniq_r_cluster:
  state_names.append(str(cluster))
state_names[:5]

X = []
labels = []
for month in months:
  for day in days:
    day_df = df[(df.day == day) & (df.month == month)]
    uniq_trips = day_df.trip_id.unique()
    for trip_id in uniq_trips:
      trip = day_df[day_df.trip_id==trip_id]
      X.append(trip.way_id.to_numpy().astype(str))
      # labels.append(['None-start']+[str(l) for l in trip.route_cluster.to_list()])
      labels.append([str(l) for l in trip.route_cluster.to_list()])
print(X[0][:5])
print(labels[0][:5])
print(len(X))
print(len(labels))

['686967969' '686957536' '686912257' '686911019' '686910206']
['1.0', '9.0', '10.0', '10.0', '10.0']
356
356


In [17]:
import random
random.seed(42)

X2 = X.copy()
labels2 = labels.copy()

temp = list(zip(X2, labels2))
random.shuffle(temp)
X2, labels2 = zip(*temp)
X2 = list(X2)
labels2 = list(labels2)

print(X[0][:5])
print(X2[0][:5])

print(labels[0][:5])
print(labels2[0][:5])

['686967969' '686957536' '686912257' '686911019' '686910206']
['817644082' '81349795' '575226821' '160728583' '160728584']
['1.0', '9.0', '10.0', '10.0', '10.0']
['-1.0', '724.0', '724.0', '724.0', '-1.0']


## train and test 10-fold cross validata

In [18]:
from collections import Counter
import pickle

# road_df = pd.read_csv(road_data_path+"/route_network_with_cluster.csv")

# cluster_df = road_df[['cluster', 'center_lat', 'center_lon']]
# cluster_df = cluster_df.drop_duplicates()
# cluster_df.rename(columns = {'cluster':'sp', 'center_lat':'lat', 'center_lon':'lon'}, inplace = True)

way_seq = df.way_id.to_list()
not_stop_mask = [True]+[(a != b) for a, b in zip(way_seq[:-1], way_seq[1:])]
df = df.loc[not_stop_mask]

way_df = df[['way_id', 'lat', 'lon']]
way_df = way_df.drop_duplicates(subset='way_id')

# loc_df = df[['trip_id', 'way_id', 'month','day']].merge(way_df,how='left')
# print(sp_seq_df.isna().any())
# loc_df

sp_list = set(map(str, way_df['way_id'].tolist()))

def score(model, seq, label):
  p = []
  prob,path = model.viterbi(seq)
  for s in path[1:]:
    p.append(s[1].name)
  
  # c = Counter(p)
  # tp = 0
  # if c[label]:
  #   tp = c[label]
  # total = sum(c.values())
  
  tp = 0
  for _state_name, _label in zip(p, label):
    if _state_name == _label:
      tp+=1
  total = len(label)


  # print("best prob : {}% ".format(np.exp(prob)*100))
  # print("beat path : {}".format(p))   
  # print("labels : {}".format(label))
  # print("sequen    : {}".format(seq))

  return tp/total

In [19]:
from pomegranate import *

total = len(X2)
fold = int(total/10)
start = 0
starts = [] # [0, 952, 1904, 2856, 3808, 4760, 5712, 6664, 7616, 8568]
for i in range(10):
  starts.append(start)
  start+=fold

accuracy_train = []
accuracy_test = []
 
for start in starts: # 10
  X_train = X2[:start]+X2[start+fold:]
  X_test = X2[start:start+fold]
  labels_train = labels2[:start]+labels2[start+fold:]
  labels_test = labels2[start:start+fold]

  # บรรทัดนี้ปลอดภัยไว้ก่อน
  uniq_sp = set()
  for seq in X_train:
    for sp in seq:
      uniq_sp.add(sp)
  uniq_label = set()
  # for label in labels_train:
  for label in labels_train:
    for uniq in list(set(label)):
      uniq_label.add(uniq)

  new_label = []
  for label2 in labels_train:
    new_label.append(['None-start']+label2)
  labels_train = new_label
  
  unseen_sp = sp_list - uniq_sp
  print(len(unseen_sp))
  X_train.append(np.array(list(unseen_sp)))
  labels_train.append(['-1'] * len(unseen_sp))
  # labels_train.append(['None-start'] + ['-1'] * len(unseen_sp))

  # uniq_label.add('-1')

  model = HiddenMarkovModel.from_samples(
      DiscreteDistribution, 
      n_components=len(uniq_label), 
      X=X_train, 
      labels=labels_train,
      algorithm='labeled',
      state_names=list(uniq_label), 
      inertia=0.001,
      max_iterations=10,
      n_jobs=-1
    ) 

  accuracy_train_percent = []
  for percent in [0.25, 0.5, 0.75, 0.9]:
    accu = 0
    for seq, label in zip(X_train, labels_train):
      sc = score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
      if sc > 0:
        # accu += sc
        accu += 1
    train_score = accu/len(X_train)
    accuracy_train_percent.append(train_score)
  accuracy_train.append(accuracy_train_percent)

  accuracy_test_percent = []
  for percent in [0.25, 0.5, 0.75, 0.9]:
    accu = 0
    for seq, label in zip(X_test, labels_test):
      sc = score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
      if sc > 0:
        # accu += sc
        accu += 1
    test_score = accu/len(X_test)
    accuracy_test_percent.append(test_score)
  accuracy_test.append(accuracy_test_percent)

  print(f'Score {start} - {start+fold}')
  print(f'Train score')
  print(f'\t25% trip traverse : {accuracy_train_percent[0]}')
  print(f'\t50% trip traverse : {accuracy_train_percent[1]}')
  print(f'\t75% trip traverse : {accuracy_train_percent[2]}')
  print(f'\t90% trip traverse : {accuracy_train_percent[3]}')
  print(f'Test score')
  print(f'\t25% trip traverse : {accuracy_test_percent[0]}')
  print(f'\t50% trip traverse : {accuracy_test_percent[1]}')
  print(f'\t75% trip traverse : {accuracy_test_percent[2]}')
  print(f'\t90% trip traverse : {accuracy_test_percent[3]}')

print('all score')
print('train score: ')
for i in range(4):
  sum = 0
  for fold in accuracy_train:
    sum += fold[i]
  print(f'\ttrip traverse {i} : {sum/10}')

print('test score: ')
for i in range(4):
  sum = 0
  for fold in accuracy_test:
    sum += fold[i]
  print(f'\ttrip traverse {i} : {sum/10}')

53


KeyError: ignored

Exception ignored in: 'pomegranate.hmm.HiddenMarkovModel._labeled_summarize'
KeyError: -1


KeyError: ignored

Exception ignored in: 'pomegranate.hmm.HiddenMarkovModel._labeled_summarize'
KeyError: -1


...

KeyError: ignored

Exception ignored in: 'pomegranate.hmm.HiddenMarkovModel._labeled_summarize'
KeyError: -1


KeyError: ignored

Exception ignored in: 'pomegranate.hmm.HiddenMarkovModel._labeled_summarize'
KeyError: -1


Score 315 - 350
Train score
	25% trip traverse : 0.5652173913043478
	50% trip traverse : 0.6987577639751553
	75% trip traverse : 0.7950310559006211
	90% trip traverse : 0.8291925465838509
Test score
	25% trip traverse : 0.34285714285714286
	50% trip traverse : 0.4
	75% trip traverse : 0.5428571428571428
	90% trip traverse : 0.5428571428571428
all score
train score: 
	trip traverse 0 : 0.5627329192546583
	trip traverse 1 : 0.6854037267080745
	trip traverse 2 : 0.7857142857142857
	trip traverse 3 : 0.8220496894409939
test score: 
	trip traverse 0 : 0.24285714285714283
	trip traverse 1 : 0.3742857142857143
	trip traverse 2 : 0.5228571428571429
	trip traverse 3 : 0.5371428571428571
